## École Polytechnique de Montréal
## Département Génie Informatique et Génie Logiciel

## INF8460 – Traitement automatique de la langue naturelle - TP2

## Objectifs d'apprentissage: 

•	Explorer les modèles d’espace vectoriel (vector space models) comme représentations distribuées de la sémantique des mots 
•	Implémenter la fréquence de co-occurrence et la PPMI
•	Comprendre différentes mesures de distance entre vecteurs de mots 
•	Explorer l’intérêt de la réduction de dimensionnalité 



## Équipe et contributions 
Veuillez indiquer la contribution effective de chaque membre de l'équipe en pourcentage et en indiquant les modules ou questions sur lesquelles chaque membre a travaillé

Cedric Sadeu (1869737): 1/3

Mamoudou Sacko (1924187): 1/3

Oumayma Messoussi (2016797): 1/3



## Support de google Colab



In [1]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!wget https://staff.fnwi.uva.nl/e.bruni/resources/MEN.tar.gz

--2020-10-04 23:15:11--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  24.1MB/s    in 5.0s    

2020-10-04 23:15:17 (16.2 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]

--2020-10-04 23:15:17--  https://staff.fnwi.uva.nl/e.bruni/resources/MEN.tar.gz
Resolving staff.fnwi.uva.nl (staff.fnwi.uva.nl)... 146.50.61.62
Connecting to staff.fnwi.uva.nl (staff.fnwi.uva.nl)|146.50.61.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90373 (88K) [application/x-gzip]
Saving to: ‘MEN.tar.gz’

MEN.tar.gz          100%[===================>]  88.25K   238KB/s    in 0.4s    

2020-10-04 23:15:18 (238 KB/s) - ‘MEN.tar.gz’ saved [90373/90373]


In [2]:
! tar -xzf aclImdb_v1.tar.gz
! tar -xzf MEN.tar.gz
! mkdir -p vsm

## Librairies externes

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [43]:
from collections import Counter, defaultdict
from itertools import chain
from bs4 import BeautifulSoup
import csv
import itertools
import numpy as np
import os
import pandas as pd
import glob
import re
import math

from nltk.corpus import stopwords as all_stopwords
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter

from scipy.stats import spearmanr
from scipy.spatial.distance import euclidean, cosine
from IPython.display import display
from sklearn.decomposition import TruncatedSVD

## Valeurs globales

In [5]:
DIRNAME_ACL =  os.path.join(os.getcwd(), "aclImdb/")
DIRNAME_MEN =  os.path.join(os.getcwd(), "MEN/")
DIRNAME_VSM =  os.path.join(os.getcwd(), "vsm")

## 1. Prétraitement (20 points)

**a)**	Le jeu de données est séparé en deux répertoires `train/`et `test`, chacun contenant eux-mêmes deux sous-répertoires `pos/` et `neg/` pour les revues positives et négatives. Un fichier `readme` décrit plus précisément les données. Commencez par lire ces données, en gardant séparées les données d'entraînement et de test. La fonction doit mettre les mots en minuscules,  supprimer les stopwords (vous devez utiliser ceux de NLTK) et afficher le nombre total de phrases d’entrainement,  le nombre total de phrases d’entrainement positives et négatives et le nombre total de phrases de test avec le nombre total de phrases de test positives et négatives ;

In [6]:
train_data = {'id': [], 'text': [], 'rating': [], 'sentiment': []}
test_data = {'id': [], 'text': [], 'rating': [], 'sentiment': []}

for name in ['train/pos', 'train/neg']:
    sentiment = name.split('/')[1]
    for file in glob.glob(DIRNAME_ACL+name+"/*.txt"):
        review_id = int(file.split("/")[-1].split(".")[0].split("_")[0])
        rating = int(file.split("/")[-1].split(".")[0].split("_")[1]) / 10
        f = open(file, encoding="utf8")
        content = f.read()
        
        train_data['id'].append(review_id)
        train_data['text'].append(content)
        train_data['rating'].append(rating)
        train_data['sentiment'].append(sentiment)
                
for name in ['test/pos', 'test/neg']:
    sentiment = name.split('/')[1]
    for file in glob.glob(DIRNAME_ACL+name+"/*.txt"):
        review_id = int(file.split("/")[-1].split(".")[0].split("_")[0])
        rating = int(file.split("/")[-1].split(".")[0].split("_")[1]) / 10
        f = open(file, encoding="utf8")
        content = f.read()
        
        test_data['id'].append(review_id)
        test_data['text'].append(content)
        test_data['rating'].append(rating)
        test_data['sentiment'].append(sentiment)

train_df = pd.DataFrame (train_data, columns = ['id', 'text', 'rating', 'sentiment'])
test_df = pd.DataFrame (test_data, columns = ['id', 'text', 'rating', 'sentiment'])

In [7]:
print(train_df.shape)
train_df.head()

(25000, 4)


,id,text,rating,sentiment
0,11003,"Listen, I don't care what anybody says, as Cyp...",1.0,pos
1,7621,"Lost, probably the best t.v series ever made. ...",1.0,pos
2,575,Great drama with all the areas covered EXCEPT ...,1.0,pos
3,5957,The title says it all.<br /><br />I'm not a fi...,0.9,pos
4,10981,"Hey now, yours truly, TheatreX, found this whi...",0.8,pos


In [8]:
print(test_df.shape)
test_df.head()

(25000, 4)


,id,text,rating,sentiment
0,569,This is a fascinating documentary about a 15 y...,0.7,pos
1,10319,"For the record, I am a Curly fan through and t...",0.8,pos
2,1192,"I Won't say anything about music, because this...",1.0,pos
3,1091,Nana Patekar once again proves that he is the ...,0.7,pos
4,4616,"At 2:37, a high school student commits suicide...",0.9,pos


In [9]:
def preprocess(review):
    soup = BeautifulSoup(review, "html.parser")
    text = soup.get_text()
    stop_words = set(all_stopwords.words('english'))
    
    tokens = nltk.word_tokenize(text.lower())
    return [w for w in tokens if w.isalpha() and not w in stop_words] 

In [10]:
%time
train_df['text'] = train_df['text'].apply(preprocess)
test_df['text'] = test_df['text'].apply(preprocess)

In [11]:
print(train_df.shape)
train_df.head()

(25000, 4)


,id,text,rating,sentiment
0,11003,"[listen, care, anybody, says, cypher, nothing,...",1.0,pos
1,7621,"[lost, probably, best, series, ever, made, sto...",1.0,pos
2,575,"[great, drama, areas, covered, except, screenl...",1.0,pos
3,5957,"[title, says, film, critic, act, like, rest, s...",0.9,pos
4,10981,"[hey, truly, theatrex, found, grubbing, videos...",0.8,pos


In [12]:
print(test_df.shape)
test_df.head()

(25000, 4)


,id,text,rating,sentiment
0,569,"[fascinating, documentary, year, old, black, l...",0.7,pos
1,10319,"[record, curly, fan, say, reality, shemp, real...",0.8,pos
2,1192,"[wo, say, anything, music, topic, deep, become...",1.0,pos
3,1091,"[nana, patekar, proves, best, actor, working, ...",0.7,pos
4,4616,"[high, school, student, commits, suicide, show...",0.9,pos


In [13]:
print("""TRAIN\n\t - le nombre total de phrases d’entrainement: %d
              \n\t - le nombre total de phrases d’entrainement positives: %d
              \n\t - le nombre total de phrases d’entrainement négatives: %d\n""" % 
      (train_df.shape[0], train_df[train_df["sentiment"] == "pos"].shape[0], 
       train_df[train_df["sentiment"] == "neg"].shape[0]))

print("""TEST\n\t - le nombre total de phrases de test: %d
              \n\t - le nombre total de phrases de test positives: %d
              \n\t - le nombre total de phrases de test négatives: %d\n""" % 
      (test_df.shape[0], test_df[test_df["sentiment"] == "pos"].shape[0], 
       test_df[test_df["sentiment"] == "neg"].shape[0]))

TRAIN
	 - le nombre total de phrases d’entrainement: 25000
              
	 - le nombre total de phrases d’entrainement positives: 12500
              
	 - le nombre total de phrases d’entrainement négatives: 12500

TEST
	 - le nombre total de phrases de test: 25000
              
	 - le nombre total de phrases de test positives: 12500
              
	 - le nombre total de phrases de test négatives: 12500



**b)**	Créez la fonction `build_voc()` qui extrait les unigrammes de l’ensemble d’entraînement et conserve ceux qui ont une fréquence d’occurrence d'au moins 5 et imprime le nombre de mots dans le vocabulaire. Sauvegardez-le dans un fichier `vocab.txt` (un mot par ligne) dans le répertoire aclImdb.

In [14]:
def build_voc(corpus, unk_cutoff=5):
    unigrams = defaultdict(int)
    for review in corpus:
        for word in review:
            unigrams[word] += 1

    print("Unigrammes: " + str(len(unigrams)))

    unigrams = {k: v for k, v in sorted(unigrams.items(), key=lambda item: item[1], reverse=True) if v >= 5}

    print("Taille du vocab (apres filtrage freq > 5): " + str(len(unigrams)))

    f = open('vocab.txt', 'w')
    for w in unigrams.keys():
        f.write(w + '\n')

    return unigrams

In [15]:
train_unigrams = build_voc(train_df['text'])

Unigrammes: 72029
Taille du vocab (apres filtrage freq > 5): 27760


## 2. Matrices de co-occurence (30 points)

Pour les matrices de cette section, vous pourrez utiliser [des array `numpy`](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html) ou des DataFrame [`pandas`](https://pandas.pydata.org/pandas-docs/stable/). 

Ressources utiles :  le [*quickstart tutorial*](https://numpy.org/devdocs/user/quickstart.html) de numpy et le guide [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html).

**a)** A partir des textes du corpus d’entrainement (neg/pos), vous devez construire une matrice de co-occurrence mot × mot M(w,w) qui contient les 5000 unigrammes les plus fréquents sous forme de **cadre panda**. Le contexte de co-occurrence est une fenêtre de +/-5 mots autour du mot cible. Le poids est la fréquence de co-occurrence simple. Sauvegardez votre matrice dans un fichier tp2_mat5.csv dans le répertoire vsm.

Attention, le mot lui même de doit pas être compté dans sa co-occurence. Exemple : 
Corpus: [ "I go to school every day by bus", "i go to theatre every night by bus"]

Co-occurence("every", fenetre=2) = [ (to, 2), (by, 2), (school, 1), (day, 1), (theatre, 1), (night, 1), (bus, 0), (every, 0), (go, 0). (i,0) ]

In [29]:
def get_neighbours(list_words, index, N, is_distance):
    result = []
    try:
        word = list_words[index]
    except:
        return result
    max_index =  index + N
    if max_index >= len(list_words):
        max_index = len(list_words) - 1
    min_index = index - N
    if min_index < 0:
        min_index = 0
    if is_distance:
        while(min_index <= max_index):
            if list_words[min_index] != word:
                result.append(((word, list_words[min_index]), 1 / (min_index - index)))
            min_index += 1
    else:
        while(min_index <= max_index):
            if list_words[min_index] != word:
                result.append((word, list_words[min_index]))
            min_index += 1
    return result

def neighbours_grapper(list_words, word, N, is_distance):
    #if list_words is empty return
    if not list_words:
        return []
    #find all indexes of word
    indexes = [i for i, w in enumerate(list_words) if w == word] 
    #if indexes is empty return
    if not indexes:
        return []
    result = []
    for i in indexes:
        result.extend(get_neighbours(list_words, i, N, is_distance))
    return result

def neighbours_corpus(corpus, N, unigramme_list, is_distance):
    result = []
    for word in unigramme_list:
        for sentence in corpus:
            sentence = re.split(r'\s', sentence)
            result.extend(neighbours_grapper(sentence, word, N, is_distance))
    return result

def dict_tuple_creator(tuple_list):
    dict_tuple = Counter(tuple_list)
    return dict(dict_tuple)

def matrix_creator(dict_tuple, unigramme_list):
    # avoir un dict avec key (mot1 , mot2) et frequence comme value
    # creer une matrice 5000 * 5000 et la remplir de zero
    # parcourir le dictionnaire:
    #   pour chaque key (mot1 , mot2): find la position du mot1 et du mot2 dans la matrice
    #                           pour avoir ces coordonnes (pos1, pos2) dans la matrice
    # mettre la value correspondante a cette key a (pos1, pos2) dans la matrice
    N = len(unigramme_list)
    list_zero = [0] * N
    mat_result = [list_zero] * N
    for key in dict_tuple.keys():
        try:
            x = unigramme_list.index(key[0])
            y = unigramme_list.index(key[1])
        except ValueError:
            continue
        mat_result[x][y] = dict_tuple[key]
    
    return mat_result

def mat_list_to_df(mat_result, unigramme_list):
    arr = np.array(mat_result)
    t_arr = arr.T
    arr_list = t_arr.tolist()
    dict_result = {}
    for i, word in enumerate(unigramme_list):
        dict_result[word] = arr_list[i]
    df = pd.DataFrame(data = dict_result, index = unigramme_list)
    return df

In [26]:
train_top_5000 = list(train_unigrams.keys())[:5000] # or read first 5000 lines from vocab.txt
train_5000_df = pd.DataFrame(train_top_5000, columns=['unigram'])
train_5000_df.shape
train_5000_df.head()

corpus = list(train_df['text'])
corpus = [' '.join(sentence) for sentence in corpus]
print(len(corpus))

25000


In [39]:
%time
corpus = [ "I go to school every day by bus", "I go to theatre every night by bus"]
vocab = ["every", "go", "night", "school", "bus"]
tuple_list = neighbours_corpus(corpus, 1, vocab, False)
dict_tuple = dict_tuple_creator(tuple_list)
mat_result = matrix_creator(dict_tuple, vocab)
df = mat_list_to_df(mat_result, vocab)
df.to_csv('tp2_mat5.csv')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
[[1, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]]


In [37]:
df.head(10)

,every,go,night,school,bus
every,1,0,1,1,0
go,1,0,1,1,0
night,1,0,1,1,0
school,1,0,1,1,0
bus,1,0,1,1,0


**b)** Calculez maintenant une matrice de cooccurrence mais en ajustant les fréquences basées sur la proximité du mot cible par exemple en les multipliant par 1/𝑑 où d est la distance en jetons (mots) de la cible. Sauvegardez votre matrice (toujours sous forme de cadre panda) dans un fichier tp2_mat5_scaled.csv dans le répertoire vsm.

In [40]:
def dict_tuple_creator_d(tuple_list):
    res = list(zip(*tuple_list))
    tup = list(res[0])
    dict_tup = Counter(tup)
    for key in dict_tup.keys():
        temp = [item[1] for item in tuple_list if item[0] == key]
        total = sum(temp)
        dict_tup[key] = total / dict_tup[key]
    return dict(dict_tup)

In [ ]:
tuple_list_d = neighbours_corpus(corpus, 5, train_top_5000, True)
dict_tuple_d = dict_tuple_creator_d(tuple_list_d)
mat_result_d = matrix_creator(dict_tuple_d, train_top_5000)
df_d = mat_list_to_df(mat_result_d, train_top_5000)
df_d.to_csv('p2_mat5_scaled.csv')

**c)**	Vous devez créer une fonction `pmi` qui prend le cadre panda de la matrice $M(w,w)$ et un paramètre boolean flag qui est à True lorsque l'on désire calculer PPMI et à False quand on veut calculer PMI. La fonction transforme la matrice en entrée en une matrice $M’(w,w)$ avec les valeurs PMI ou PPMI selon la valeur du paramètre booléen. La fonction retourne le nouveau cadre panda correspondant. 

Pour une matrice  $X_{m \times n}$:


$$\textbf{colsum}(X, j) = \sum_{i=1}^{m}X_{ij}$$

$$\textbf{sum}(X) = \sum_{i=1}^{m}\sum_{j=1}^{n} X_{ij}$$

$$\textbf{expected}(X, i, j) = 
\frac{
  \textbf{rowsum}(X, i) \cdot \textbf{colsum}(X, j)
}{
  \textbf{sum}(X)
}$$


$$\textbf{pmi}(X, i, j) = \log\left(\frac{X_{ij}}{\textbf{expected}(X, i, j)}\right)$$

$$\textbf{ppmi}(X, i, j) = 
\begin{cases}
\textbf{pmi}(X, i, j) & \textrm{if } \textbf{pmi}(X, i, j) > 0 \\
0 & \textrm{otherwise}
\end{cases}$$


In [48]:
def probability_matrix(mat_Result):
    total = 0
    for line in mat_Result:
        total += sum(line)
    arr = np.array(mat_Result)
    arr_prob = np.divide(arr, total)
    return arr_prob.tolist()

def ppmi_pmi(df_Result, is_ppmi):
    temp = df_Result.to_numpy()
    mat_Result = list(temp.tolist())
    mat_prob = probability_matrix(mat_Result)
    print(mat_prob)
    arr = np.array(mat_prob)
    sum_arr_column = arr.sum(axis=0)
    sum_arr_line = arr.sum(axis=1)

    with np.errstate(divide='ignore'):
      for i, line in enumerate(mat_prob):
          for j, item in enumerate(line):
              if item == 0:
                  mat_prob[i][j] = 0
              elif is_ppmi:         
                  mat_prob[i][j] = max(math.log(item / (sum_arr_column[j]*sum_arr_line[i]), 2), 0)
              else:
                  mat_prob[i][j] = math.log(item / (sum_arr_column[j]*sum_arr_line[i]), 2)
    
    return mat_list_to_df(mat_prob, list(df_Result.columns))


#def pmi(df, flag=True):

**d)** Créer les matrice PMIs et PPMIs en vous basant sur les deux matrices que vous avez déjà créée	Sauvegardez vos matrices dans un fichier tp2_mat5<\_scaled>_{pmi|ppmi}.csv toujours dans le répertoire vsm. 

(votre nom de fichier doit contenir "_scaled" s'il est formé à partir Mww_scaled et "pmi" si le flag est false "ppmi" sinon) 

In [49]:
result_ppmi_df = ppmi_pmi(df, True)
result_ppmi_df.to_csv('tp2_mat5_ppmi.csv')
result_pmi_df = ppmi_pmi(df, False)
result_pmi_df.to_csv('tp2_mat5_pmi.csv')

# result_ppmi_sc_df = ppmi_pmi(df_d, True)
# result_ppmi_sc_df.to_csv('tp2_mat5_scaled_ppmi.csv')
# result_pmi_sc_df = ppmi_pmi(df_d, False)
# result_pmi_sc_df.to_csv('tp2_mat5_scaled_pmi.csv')

[[0.06666666666666667, 0.0, 0.06666666666666667, 0.06666666666666667, 0.0], [0.06666666666666667, 0.0, 0.06666666666666667, 0.06666666666666667, 0.0], [0.06666666666666667, 0.0, 0.06666666666666667, 0.06666666666666667, 0.0], [0.06666666666666667, 0.0, 0.06666666666666667, 0.06666666666666667, 0.0], [0.06666666666666667, 0.0, 0.06666666666666667, 0.06666666666666667, 0.0]]
[[0.06666666666666667, 0.0, 0.06666666666666667, 0.06666666666666667, 0.0], [0.06666666666666667, 0.0, 0.06666666666666667, 0.06666666666666667, 0.0], [0.06666666666666667, 0.0, 0.06666666666666667, 0.06666666666666667, 0.0], [0.06666666666666667, 0.0, 0.06666666666666667, 0.06666666666666667, 0.0], [0.06666666666666667, 0.0, 0.06666666666666667, 0.06666666666666667, 0.0]]


## 3. Test de PPMI (20 points)

Pour le test des matrices de cooccurrences, nous allons comparer deux mesures de distance entre deux vecteurs, la distance euclidienne et la distance cosinus provenant du module [scipy.spatial.distance](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

**Distance Euclidienne**

La distance euclidienne entre deux vecteurs $u$ et $v$ de dimension $n$ est

$$\textbf{euclidean}(u, v) = 
\sqrt{\sum_{i=1}^{n}|u_{i} - v_{i}|^{2}}$$

En deux dimensions, cela correspond à la longueur de la ligne droite entre deux points.

**Distance Cosinus**


La distance cosinus entre deux vecteurs $u$ et $v$ de dimension $n$ s'écrit :

$$\textbf{cosine}(u, v) = 
1 - \frac{\sum_{i=1}^{n} u_{i} \cdot v_{i}}{\|u\|_{2} \cdot \|v\|_{2}}$$

Le terme de droite dans la soustraction mesure l'angle entre $u$ et $v$; on l'appelle la *similarité cosinus* entre $u$ et $v$.

\\

**a)**	Implémentez la fonction voisins(mot, pd, distance) qui prend un mot en entrée et une métrique de distance et qui retourne les n mots les plus similaires selon la mesure. Pour un mot w, elle ordonne tous les mots du vocabulaire en fonction de leur distance de w en utilisant la métrique de distance distance (par défaut: cosine)sur le vsm pd. Les mesures de distance à tester sont : la distance Euclidienne et la distance cosinus implantées ci-dessus.

In [ ]:
def voisins(word, df, n, distfunc=cosine):
    assert distfunc.__name__ == 'cosine' or distfunc.__name__ == 'euclidean', "distance metric not supported"
    assert word in df.columns, "word not in df"
    order = True if distfunc.__name__ == 'euclidean' else False

    closest = {}
    for w in df.columns:
        if w == word:
            continue
        distance = distfunc(df[word], df[w])
        closest[w] = distance
    closest = {k: v for k, v in sorted(closest.items(), key=lambda item: item[1], reverse=order)}

    return list(closest.keys())[:n], list(closest.values())[:n]

In [ ]:
Mtest = pd.DataFrame([[2.0, 10.0, 14.0], [6.0, 14.0, 9.0]], columns=['excellent', 'bon', 'decevant'])
print(Mtest.head())
print(voisins('bon', Mtest, 5, euclidean))
print(voisins('bon', Mtest, 5, cosine))

   excellent   bon  decevant
0        2.0  10.0      14.0
1        6.0  14.0       9.0
(['excellent', 'decevant'], [11.313708498984761, 6.4031242374328485])
(['excellent', 'decevant'], [0.044220991278050015, 0.0710423905158224])


**b)** En utilisant le cadre panda associé aux matrices Mww et Mww scaled, trouvez les 5 mots les plus similaires au mot « beautiful » et affichez-les, pour chacune des deux distances.

In [ ]:
print(voisins('beautiful', Mww, 5, euclidean))
print(voisins('beautiful', Mww_scaled, 5, cosine))

**c)** En utilisant les cadres panda associés aux matrices PMIs, trouvez les 5 mots les plus similaires au mot « beautiful » et affichez-les, pour chacune des deux distances.

In [ ]:
print(voisins('beautiful', Mpmi, 5, euclidean))
print(voisins('beautiful', Mpmi_scaled, 5, cosine))

**d)** En utilisant les cadres panda associés aux matrices PPMIs, trouvez les 5 mots les plus similaires au mot
« beautiful » et affichez-les, pour chacune des deux distances. 

In [ ]:
print(voisins('beautiful', Mppmi, 5, euclidean))
print(voisins('beautiful', Mppmi_scaled, 5, cosine))

**e)** Que constatez-vous entre la différence de performance de la distance euclidienne et la distance cosinus ? Que constatez-vous entre les différents types de matrices de cooccurrence ?

## 4.	Réduction de dimensionnalité (20 points)

**a)** Ecrivez une fonction lsa qui prend en entrée un cadre panda pd (qui contient votre matrice / vsm) et un paramètre K (qui indique le nombre de dimensions finales), et qui applique LSA avec ce paramètre k sur la matrice et retourne le vsm réduit sous forme de cadre panda.

In [ ]:
mots = ['red', 'blue', 'green', 'yellow', 'orange', 'black']
data = np.array([ [1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0], [1, 0, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1]])
vsm = pd.DataFrame(data,  columns=mots)
PMIs = pd.DataFrame(data,  columns=mots)
PPMIs = pd.DataFrame(data, columns=mots)

In [ ]:
def lsa(df, k):
    U, s, V = np.linalg.svd(df, full_matrices=False)
    S = np.diag(s[:k])
    Ck = np.dot(U[:, :k], S)
    print(Ck)
    return pd.DataFrame(Ck)

display(lsa(vsm, 3))

**b)** Exécutez lsa sur les cadres panda associés à vos matrices Mww et Mww_scaled avec une dimension k=100

In [ ]:
PMIs = lsa(PMIs, 100)
PPMIs = lsa(PPMIs, 100)

**c)** En utilisant les matrices de co-occurrence (de base et scalés) réduites avec LSA, trouvez les 5 mots les plus similaires au mot « beautiful » selon la distance cosinus et affichez-les.

d) En utilisant les matrices PMIs et PPMIs réduites avec lsa, trouvez les 5 mots les plus similaires au mot « beautiful » selon la distance cosinus et affichez-les

In [ ]:
print(voisins('beautiful', PMIs, cosine)[:5])
print(voisins('beautiful', PPMIs, cosine)[:5])

**e)** En utilisant sklearn.decomposition.TruncatedSVD, créez les matrices réduites à partir des mêmes matrices que celles de la question précédentes (la matrice pmi et la matrice pmi_scaled) Puis tester ces nouvelles matrices LSA pour trouver les 5 mots les plus similaires au mot « beautiful » 

Ici aussi, nous voulons aussi obtenir des matrices de dimension k=100

In [ ]:
PMIs = pd.DataFrame(data, columns=mots)
PPMIs = pd.DataFrame(data, columns=mots)

svd_model = TruncatedSVD(n_components=5, algorithm='randomized', n_iter=100, random_state=122)

PMIs = svd_model.fit_transform(PMIs)
PPMIs = svd_model.fit_transform(PPMIs)

display(PMIs)
display(PPMIs)

f) Commentez vos résultats

## 5. Évaluation (10 points)

Il est temps d’évaluer l’intérêt de nos modèles de vecteurs. Nous allons pour cela utiliser un ensemble de données de similarité de mots (relatedness) The MEN Test Collection, qui se trouve dans le répertoire test. L’ensemble de données contient une paire de mots avec un score de similarité attribué par des humains. En d’autres termes, une ligne (un exemple) de l’ensemble de données est de la forme : \<mot_1> \<mot_2> \<score>.

Pour aligner les distances obtenues avec vos métriques, ce score est converti en nombre réel négatif par la fonction read_test_dataset que vous avez dans le squelette du TP.

La métrique d’évaluation est le coefficient de corrélation de Spearman 𝜌 entre les scores humains et vos distances (voir https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient). 

Nous allons maintenant évaluer les différents vsm obtenus sur l'ensemble de données: MEN_dataset.


#### Fonctions pour lire le jeu de données MEN

In [ ]:
def read_test_dataset(
        src_filename,
        delimiter=','):
    with open(src_filename) as f:
        reader = csv.reader(f, delimiter=delimiter)
        for row in reader:
            w1 = row[0].strip().lower()
            w2 = row[1].strip().lower()
            score = row[2]
            score = -float(score)
            yield (w1, w2, score)

In [ ]:
# Retourne un itérable sur le jeu de données MEN
def men_dataset():
    src_filename = os.path.join(
        DIRNAME_MEN, 'MEN_dataset_natural_form_full')
    return read_test_dataset(
        src_filename, delimiter=' ')

In [ ]:
def evaluate(ds, df, distfunc=cosine):
    """
    ds : iterator
       retourne des tuples (word1, word2, score).

    df : pd.DataFrame
        le modèle vsm à évaluer

    distfunc : la mesure de distance entre vecteurs
  
    Retour: le coefficient de correlation de Spearman entre les scores de l'ensemble de données de test 
    et celui du modele vsm qui se présente sous la forme d'un cadre Panda pd avec les colonnes
    ['word1', 'word2', 'score', 'distance'].
    """
    data = []
    for w1, w2, score in ds():
        d = {'word1': w1, 'word2': w2,'score': score}
        if w1 not in df.index or w2 not in df.index:
            continue
        else:
            w1 = df.loc[w1]
            w2 = df.loc[w2] 
        d['distance'] = distfunc(w1, w2)
        data.append(d)

    data = pd.DataFrame(data)
    rho, pvalue = spearmanr(data['score'].values, b=data['distance'].values)
    return rho, data

**a)**	Testez chacun de vos modèles vsm (Matrice de base, matrice scalée, les PMIs et PPMIs et toutes les matrices LSA (de base, scalée, pmi, ppmi) en appelant la fonction evaluate avec les deux mesure de distance (euclidienne et cosinus) et affichez vos résultats dans une seule table.

**b)**	Commentez vos résultats d'évaluation